In [1]:
# Erklärung zur Extraktion der Nutzer*innenkommentare:
#
# Üblicherweise wird eine sogenannte "API" ("application programming interface", oder auf deutsch: Programmier-
# schnittstelle) verwendet, um Kommentare aus sozialen Netzwerken zu extrahieren. Nach dem Cambridge Analy-
# tica-Skandal 2018 wurde die Facebook-eigene "Graph API" aus guten Gründen so stark reglementiert, dass im
# Rahmen dieser Arbeit kein direkter Zugriff auf die Daten der Facebook-Kommentare unter den Live-Videos des
# WDR möglich ist, auch wenn diese Kommentare öffentlich für jeden einsehbar sind.
# 
# Um diesem Problem pragmatisch zu begegnen, wurden die Facebook Live-Kommentare mit folgendem Verfahren extra-
# hiert, welches in einzelnen Schritten manuell, in anderen automatisiert abläuft.
#
# 1.) Das Video zum Live-Stream wurde im Browser geöffnet.
#
# 2.) Alle Kommentare im Kommentarbereich wurden mit der Funktion "nach Reihenfolge anzeigen" chronologisch
#     sortiert und vollständig lesbar gemacht (mit Klicks auf alle Schaltflächen, die verborgenen Text
#     sichtbar machen (wie z.B. "mehr anzeigen", "2 Antworten sehen" etc.).
#
# 3.) Die im Browser nun dargestellte Webseite wurde anschließend als HTML-Dokument lokal auf dem Computer
#     gespeichert (im Browser Firefox mit der Option "Seite speichern unter...").
#
# 4.) Das HTML-Dokument wurde mit dem HTML-Editor "Brackets" so bearbeitet, dass nur der Kommentarbereich
#     im Dokument enthalten bleibt. (Da ansonsten auch sämtliche private Unterhaltungen der Person, welche
#     die Daten extrahiert hat, in den HTML-Dokumenten enthalten und diese Dateien nicht zur Dokumen-
#     tation geeignet gewesen wären.) Diese Dateien befinden sich auch auf dem der Bachelorarbeit bei-
#     gelegten Datenträger.
#
# 5.) Die Kommentare wurden mit dem folgenden Python-Programm aus den HTML-Dokumenten extrahiert und in einen
#     gemeinsamen Datensatz abgelegt.

from bs4 import BeautifulSoup
import urllib3
import re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pytz
from pytz import timezone

excel_import =  pd.read_excel('sendungen_mit_timestamps.xlsx', 'Tabelle1', na_values=['NA'])
sendungen = pd.DataFrame(excel_import).values

sendungengesamt = []
usergesamt = []

for sendung in sendungen:
    sendung_nr = sendung[0]
    sendung_titel = sendung[2]
    video_ende_stempel = sendung[5]
    video_start_u = sendung[17]
    video_start_utc = sendung[18]
    video_start_mez = sendung[19]
    video_start_stempel = sendung[20]
    video_start_s = sendung[21]
    video_start_s = sendung[21]
    video_ende_u = sendung[22]
    video_ende_utc = sendung[23]
    video_ende_mez = sendung[24]
    video_ende_stempel = sendung[25]
    video_ende_s = sendung[26]
    sendung_start_u = sendung[27]
    sendung_start_utc = sendung[28]
    sendung_start_mez = sendung[29]
    sendung_start_stempel = sendung[30]
    sendung_start_s = sendung[31]
    sendung_ende_u = sendung[32]
    sendung_ende_utc = sendung[33]
    sendung_ende_mez = sendung[34]
    sendung_ende_stempel = sendung[35]
    sendung_ende_s = sendung[36]   

    #Liste mit Nutzer*innen sowie Nutzer*Innen-Nr.
    user_liste = {}
    user_zaehler = 1

    url = sendung_nr+".html"
    page=open(url, mode='r', encoding='utf8')
    soup = BeautifulSoup(page.read())

    # Erstellen einer Liste, in der die HTML-Pfade aller Nutzer-Kommentare abgelegt werden:
    kommentare = soup.findAll(attrs={"role":"article"})

    # Erstellen einer zweiten Liste, in der die Kommentare als Fälle eines Datensatzes abgelegt werden sollen:
    liste_kommentare = []

    # Jeder Kommentar wird als eine Liste von Variablenausprägungen behandelt. Mit der folgenden Schleife wer-
    # den für jeden Kommentar die dazugehörigen Variablenausprägungen aus dem HTML-Dokument extrahiert und in
    # der Liste des Kommentars abgelegt.

    # Einrichtung eines Zählers für die Reihenfolge der Kommentare im Kommentarbereich:
    kommentar_nr = 1

    for kommentar in kommentare:
        # Prüfung, ob das Objekt, was auf Basis des Attributs {"role":"article"} herausgezogen wurde,
        # wirklich ein Kommentar ist. Dazu wird untersucht, ob das Objekt einen Zeitstempel im Video besitzt:
        if kommentar.find(attrs={"class":"UFICommentActorName"})!=None:

            eintrag_kommentar = []

            # 1.) id --> ID Kommentar
            id = sendung_nr+"_k"+str(kommentar_nr).zfill(4)

            # 2.) sendung_nr --> Nummer zur Identifikation der Sendung, bereits oben definiert
            # 3.) sendung_titel --> Titel der Sendung, bereits oben definiert
            # 4.) kommentar_nr --> Position des Kommentars im Thread, bereits oben definiert
            
            # 5.) user_nr --> Nummer zur Nutzer*innen-Identifikation, sowie:
            # 6.) user_name --> Nutzer*innen-Name
            
            user_name = kommentar.find(attrs={"class":"UFICommentActorName"}).text
            if user_name not in user_liste:
                user_nr = sendung_nr+"_u"+str(user_zaehler).zfill(3)
                user_liste.update({user_name:user_nr})
                user_zaehler = user_zaehler+1
                userfuerliste = []
                userfuerliste.append(user_nr)
                userfuerliste.append(user_name)
                postings = 0
                userfuerliste.append(postings)
                usergesamt.append(userfuerliste)
            else: user_nr = user_liste[user_name]
            for eintrag in usergesamt:
                if user_nr == eintrag[0]:
                    eintrag[2] = int(eintrag[2])+1
                    
            # 7.) postings --> Anzahl Postings durch Nutzer*in in der jeweiligen Sendung
            # Wird weiter unten zum Fall hinzugefügt, nachdem alle Kommentare extrahiert wurden
            # und die Anzahl der Postings feststeht.
                    
            # 8.) text --> Text des Kommentars
            text = kommentar.find(attrs={"class":"UFICommentBody"}).text
            
            # 9.) textlen --> Länge des Kommentartextes in Zeichen
            textlen = len(text)
            
            # 10.) u --> Zeitpunkt des Postings, Unixzeit (welche von Facebook im HTML-Code verwendet wird.
            # Es handelt sich hier um die vergangenen Sekunden seit Donnerstag, dem 1. Januar 1970, 00:00
            # Uhr UTC)
            u = kommentar.find(attrs={"class":"UFISutroCommentTimestamp livetimestamp"})["data-utime"]
            utc = pytz.utc
            utc.zone
            fmt = '%Y-%m-%d %H:%M:%S %Z%z'   
            kommentar_utc = datetime.utcfromtimestamp(int(u))
            video_start_utc = utc.localize(kommentar_utc)           
            
            # 11.) mez --> Zeitpunkt des Postings, Mitteleuropäische Zeit
            de = timezone('Europe/Berlin')
            de.zone
            mez = str(video_start_utc.astimezone(de))

            # 12.) stempel --> Zeitpunkt des Postings, Zeitstempel im Video
            zeitstempel_roh = kommentar.find(attrs={"role":"button"}).text
            if len(zeitstempel_roh) == 4:
                stempel = "00:0"+str(zeitstempel_roh)
            if len(zeitstempel_roh) == 5:
                stempel = "00:"+str(zeitstempel_roh)
            if len(zeitstempel_roh) == 7:
                stempel = "0"+str(zeitstempel_roh)
                
            # 13.) s --> Zeitpunkt des Postings, Sekunde s im Videostream
            stunden_video = int(stempel[0:2])
            minuten_video = int(stempel[3:5])
            sekunden_video = int(stempel[6:8])
            s = sekunden_video+minuten_video*60+stunden_video*3600
            
            
            # Sendungsbezogene Variablen, welche mit dem Programm "sendungszeiten_berechnen" bereits
            # berechnet und in "sendungen_mit_timestamps.xlsx" abgelegt sind:
            # 14.) video_start_u --> Beginn des Videostreams, Unixzeit
            # 15.) video_start_mez --> Beginn des Videostreams, Mitteleuropäische (Sommer-)Zeit
            # 16.) video_start_stempel --> Beginn des Videostreams, Zeitstempel im Video
            # 17.) video_start_s --> Beginn des Videostreams, Sekunde s im Videostream
            # 18.) video_ende_u --> Ende des Videostreams, Unixzeit
            # 19.) video_ende_mez --> Ende des Videostreams, Mitteleuropäische (Sommer-)Zeit
            # 20.) video_ende_stempel --> Ende des Videostreams, Zeitstempel im Video (bereits oben definiert)
            # 21.) video_ende_s --> Ende des Videostreams, Sekunde s im Videostream
            # 22.) sendung_start_u --> Beginn der Sendung, Unixzeit
            # 23.) sendung_start_mez --> Beginn der Sendung, Mitteleuropäische (Sommer-)Zeit
            # 24.) sendung_start_stempel --> Beginn der Sendung, Zeitstempel im Video
            # 25.) sendung_start_s --> Beginn der Sendung, Sekunde s im Videostream
            # 26.) sendung_ende_u --> Ende der Sendung, Unixzeit
            # 27.) sendung_ende_mez --> Ende der Sendung, Mitteleuropäische (Sommer-)Zeit
            # 28.) sendung_ende_stempel --> Ende der Sendung, Zeitstempel im Video
            # 29.) sendung_ende_s --> Ende der Sendung, Sekunde s im Videostream

            # Hinzufügen der erstellten Variablen:
            eintrag_kommentar.append(id)
            eintrag_kommentar.append(sendung_nr)
            eintrag_kommentar.append(sendung_titel)
            eintrag_kommentar.append(kommentar_nr)
            eintrag_kommentar.append(user_nr)
            eintrag_kommentar.append(user_name)
            eintrag_kommentar.append(text)
            eintrag_kommentar.append(textlen)
            eintrag_kommentar.append(u)
            eintrag_kommentar.append(mez)
            eintrag_kommentar.append(stempel)
            eintrag_kommentar.append(s)
            eintrag_kommentar.append(video_start_u)
            eintrag_kommentar.append(video_start_mez)
            eintrag_kommentar.append(video_start_stempel)
            eintrag_kommentar.append(video_start_s)
            eintrag_kommentar.append(video_ende_u)
            eintrag_kommentar.append(video_ende_mez)
            eintrag_kommentar.append(video_ende_stempel)
            eintrag_kommentar.append(video_ende_s)
            eintrag_kommentar.append(sendung_start_u)
            eintrag_kommentar.append(sendung_start_mez)
            eintrag_kommentar.append(sendung_start_stempel)
            eintrag_kommentar.append(sendung_start_s)
            eintrag_kommentar.append(sendung_ende_u)
            eintrag_kommentar.append(sendung_ende_mez)
            eintrag_kommentar.append(sendung_ende_stempel)
            eintrag_kommentar.append(sendung_ende_s)
            
            # Erhöhen des Zählers "kommentar_nr" um 1:
            kommentar_nr=kommentar_nr+1

            # Jetzt sind alle Informationen zum Kommentar gesammelt
            # und der Kommentar kann in die Liste abgelegt werden:
            liste_kommentare.append(eintrag_kommentar)
            sendungengesamt.append(eintrag_kommentar)
    
    # Anschließend wird noch die Anzahl der Postings pro Nutzer*in (Variable 7)
    # in der Sendung als nutzer*innenbezogene Variable hinzugefügt
    for eintrag_kommentar in liste_kommentare:
        for eintrag in usergesamt:
            if eintrag_kommentar[4] == eintrag[0]:
                eintrag_kommentar.insert(6, int(eintrag[2]))
    
    # Jetzt, wo die Schleife für jeden Kommentar die notwendigen Informationen aus dem HTML-Dokument extra-
    # hiert hat, soll auch ein "richtiger" Datensatz mit Variablen erstellt und als Datei abgelegt werden.
    
    datensatz_xlsx = pd.DataFrame(liste_kommentare, columns=('id', 'sendung_nr', 'sendung_titel',
                                                             'kommentar_nr', 'user_nr', 'user_name', 'postings',
                                                             'text', 'textlen', 'u', 'mez', 'stempel', 
                                                             's', 'video_start_u', 'video_start_mez', 
                                                             'video_start_stempel', 'video_start_s', 
                                                             'video_ende_u', 'video_ende_mez', 
                                                             'video_ende_stempel', 'video_ende_s', 
                                                             'sendung_start_u', 'sendung_start_mez', 
                                                             'sendung_start_stempel', 'sendung_start_s', 
                                                             'sendung_ende_u', 'sendung_ende_mez', 
                                                             'sendung_ende_stempel', 'sendung_ende_s'))
    datensatz_xlsx.to_excel("Export_"+sendung_nr+".xlsx", sheet_name='Tabelle1', index=False)
    print("Datensatz für Sendung "+sendung_nr+" wurde erstellt.")
    datensatz_xlsx
    
# Erstellung eines Datensatzes mit allen Kommentaren zu allen Sendungen
sendungengesamt_xlsx = pd.DataFrame(sendungengesamt, columns=('id', 'sendung_nr', 'sendung_titel',
                                                         'kommentar_nr', 'user_nr', 'user_name', 'postings',
                                                         'text', 'textlen', 'u', 'mez', 'stempel', 
                                                         's', 'video_start_u', 'video_start_mez', 
                                                         'video_start_stempel', 'video_start_s', 
                                                         'video_ende_u', 'video_ende_mez', 
                                                         'video_ende_stempel', 'video_ende_s', 
                                                         'sendung_start_u', 'sendung_start_mez', 
                                                         'sendung_start_stempel', 'sendung_start_s', 
                                                         'sendung_ende_u', 'sendung_ende_mez', 
                                                         'sendung_ende_stempel', 'sendung_ende_s'))
sendungengesamt_xlsx.to_excel("Export_Sendungen_Gesamt.xlsx", sheet_name='Tabelle1', index=False)
print("Datensatz für alle Sendungen wurde erstellt.")

# Erstellung eines Datensatzes mit allen User*innen aus allen Sendungen mit der Anzahl Postings innerhalb der Sendungen
usergesamt_xlsx = pd.DataFrame(usergesamt, columns=('user_nr','user_name','postings'))
usergesamt_xlsx.to_excel("Export_User_Gesamt.xlsx", sheet_name='Tabelle1', index=False)
print("Datensatz mit der Userliste wurde erstellt.")
sendungengesamt_xlsx

Datensatz für Sendung s01 wurde erstellt.
Datensatz für Sendung s02 wurde erstellt.
Datensatz für Sendung s03 wurde erstellt.
Datensatz für Sendung s04 wurde erstellt.
Datensatz für Sendung s05 wurde erstellt.
Datensatz für Sendung s06 wurde erstellt.
Datensatz für Sendung s07 wurde erstellt.
Datensatz für Sendung s08 wurde erstellt.
Datensatz für alle Sendungen wurde erstellt.
Datensatz mit der Userliste wurde erstellt.


,id,sendung_nr,sendung_titel,kommentar_nr,user_nr,user_name,postings,text,textlen,u,...,video_ende_stempel,video_ende_s,sendung_start_u,sendung_start_mez,sendung_start_stempel,sendung_start_s,sendung_ende_u,sendung_ende_mez,sendung_ende_stempel,sendung_ende_s
0,s01_k0001,s01,Deutschland - ein zerrissenes Land?,1,s01_u001,Barbara Mollenhauer,1,👋,1,1538071998,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
1,s01_k0002,s01,Deutschland - ein zerrissenes Land?,2,s01_u002,Alexander Brauer,1,Hi,2,1538071999,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
2,s01_k0003,s01,Deutschland - ein zerrissenes Land?,3,s01_u003,Silke Sieksmeier,1,hey,3,1538072009,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
3,s01_k0004,s01,Deutschland - ein zerrissenes Land?,4,s01_u004,Sylvia Breier,1,😂😂😂,3,1538072010,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
4,s01_k0005,s01,Deutschland - ein zerrissenes Land?,5,s01_u005,Ursula Wolf,2,👍,1,1538072015,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
5,s01_k0006,s01,Deutschland - ein zerrissenes Land?,6,s01_u006,Bettina Mertens,1,👍,1,1538072022,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
6,s01_k0007,s01,Deutschland - ein zerrissenes Land?,7,s01_u007,Hubert Wi,1,hi,2,1538072023,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
7,s01_k0008,s01,Deutschland - ein zerrissenes Land?,8,s01_u008,Re Gine,1,Hallo,5,1538072024,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
8,s01_k0009,s01,Deutschland - ein zerrissenes Land?,9,s01_u009,Veit Lichtenegger,1,👋,1,1538072025,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504
9,s01_k0010,s01,Deutschland - ein zerrissenes Land?,10,s01_u010,Frank M. Oeppert,1,👋Hallo,6,1538072034,...,01:32:21,5541,1538072112,2018-09-27 20:15:12+02:00,00:03:05,185,1538077431,2018-09-27 21:43:51+02:00,01:31:44,5504


In [1]:
graphsendung = "s08"
plotx=[]
ploty=[]

for sendung in sendungengesamt:
    if sendung[1] == graphsendung:
        plotx.append(sendung[4])
        ploty.append(sendung[9])
plt.plot(ploty, plotx, "ro")

NameError: name 'sendungengesamt' is not defined